## 0. 모듈임포트
-----

In [1]:
import os
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

## 1. 전처리
-----

### 유저별 평가 데이터 불러오기

In [390]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


### rating이 3점 이상인 데이터만 남기기

In [391]:
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


### rating 컬럼 이름 count로 변경

In [392]:
ratings.rename(columns={'rating':'count'}, inplace=True)

### 영화별 장르 데이터 불러오기

In [424]:
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies = movies.dropna()
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [394]:
ratings = ratings.merge(movies).drop('genre', axis=1)
ratings_temp = ratings.copy()

In [395]:
ratings = ratings_temp

In [396]:
ratings['title'] = ratings['title'][:].str.slice(start=0, stop=-7)
ratings = ratings.dropna()
ratings

,user_id,movie_id,count,timestamp,title
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest
...,...,...,...,...,...
836473,5851,3607,5,957756608,One Little Indian
836474,5854,3026,4,958346883,Slaughterhouse
836475,5854,690,3,957744257,"Promise, The (Versprechen, Das)"
836476,5938,2909,4,957273353,"Five Wives, Three Secretaries and Me"


## 2. 분석
-----

### 사용자와 영화의 유니크한 개수 확인

In [397]:
num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()
print(f'num_user: {num_user}\nnum_movie: {num_movie}')

num_user: 6039
num_movie: 3628


### 영화 인기순위 30위 뽑기

#### 각 영화별 별점 평가 개수와 별점의 평균 구하기

In [398]:
grouped = ratings['count'].groupby(ratings['movie_id'])
movie_count = grouped.count().sort_values(ascending=False).to_frame()
movie_mean = grouped.mean().sort_values(ascending=False).to_frame()

#### movie_score 데이터프레임 생성

In [399]:
movie_score = movie_count
movie_score['mean'] = movie_mean['count']
movie_score.rename(columns = {'count':'rating_count'}, inplace =True)
movie_score.head()

,rating_count,mean
movie_id,,
2858,3211,4.499844
260,2910,4.528522
1196,2885,4.384055
1210,2716,4.161635
2028,2561,4.431082


 #### 상위 30개 구하기
 * 평점의 평균이 4점 이상인 것중 평가개수 상위 30개를 선택

In [400]:
top_30 = movie_score[movie_score['mean'] >= 4.0]
top_30 = top_30[top_30['rating_count'] >= 1700][:30]
top_30 = top_30.sort_values('mean', ascending=False).reset_index()
top_30.merge(movies)

,movie_id,rating_count,mean,title,genre
0,858,2167,4.598523,"Godfather, The (1972)",Action|Crime|Drama
1,318,2194,4.596627,"Shawshank Redemption, The (1994)",Drama
2,527,2257,4.571112,Schindler's List (1993),Drama|War
3,260,2910,4.528522,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
4,1198,2473,4.520421,Raiders of the Lost Ark (1981),Action|Adventure
5,2858,3211,4.499844,American Beauty (1999),Comedy|Drama
6,2762,2385,4.487631,"Sixth Sense, The (1999)",Thriller
7,2571,2434,4.479458,"Matrix, The (1999)",Action|Sci-Fi|Thriller
8,296,2030,4.459606,Pulp Fiction (1994),Crime|Drama
9,593,2498,4.441954,"Silence of the Lambs, The (1991)",Drama|Thriller


## 3. 내가 선호하는 영화 ratings에 추가
-----

### 선호하는 영화 5개의 평점 추가

#### user_id의 최대값 조회

In [401]:
ratings['user_id'].max()

6040

#### 추가할 영화 평점 데이터 생성

In [402]:
favorites5 = [ 
    {'user_id':6041, 'movie_id':858, 'count':5, 'timestamp': 218191713, 'title':'Godfather, The'},
    {'user_id':6041, 'movie_id':2571, 'count':5, 'timestamp': 218191713, 'title':'Matrix, The'},
    {'user_id':6041, 'movie_id':1, 'count':5, 'timestamp': 218191713, 'title':'Toy Story'},
    {'user_id':6041, 'movie_id':589, 'count':5, 'timestamp': 218191713, 'title':'Terminator 2: Judgment Day'},
    {'user_id':6041, 'movie_id':3578, 'count':5, 'timestamp': 218191713, 'title':'Gladiator'} 
]

#### 내 평점 추가

In [403]:
for rating in favorites5:
    ratings = ratings.append(rating, ignore_index=True)
ratings[-10:]

,user_id,movie_id,count,timestamp,title
836473,5851,3607,5,957756608,One Little Indian
836474,5854,3026,4,958346883,Slaughterhouse
836475,5854,690,3,957744257,"Promise, The (Versprechen, Das)"
836476,5938,2909,4,957273353,"Five Wives, Three Secretaries and Me"
836477,5948,1360,5,1016563709,Identification of a Woman (Identificazione di ...
836478,6041,858,5,218191713,"Godfather, The"
836479,6041,2571,5,218191713,"Matrix, The"
836480,6041,1,5,218191713,Toy Story
836481,6041,589,5,218191713,Terminator 2: Judgment Day
836482,6041,3578,5,218191713,Gladiator


## 4. csr matrix 생성
-----

In [404]:
num_user = ratings['user_id'].nunique()
num_movie = movies['title'].nunique()
print(f'num_user: {num_user}\nnum_movie: {num_movie}')

num_user: 6040
num_movie: 3883


In [405]:
ratings = ratings.rename(columns={'count':'counts'})

* csr_matrix() 함수 호출시 rating.count로 사용하면 count함수를 호출하게 되어 에러가 발생한다. 
* 따라서 컬럼명을 counts로 변경해주었다.

In [406]:
movie_to_idx = {v:k for k,v in enumerate(ratings['title'].unique())}
user_to_idx = {v:k for k,v in enumerate(ratings['user_id'].unique())}

In [408]:
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):  
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   
else:
    print('user_id column indexing Fail!!')

user_id column indexing OK!!


In [409]:
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('title column indexing OK!!')
    ratings['movie_id'] = temp_movie_data 
else:
    print('title column indexing Fail!!')

title column indexing OK!!


> **movie_id를 title컬럼의 유니크 값을 기준으로 부여한 인덱스로 변경한 이유**   
  검색시에는 문자열이 title로 검색해야하는데, src matrix를 생성할 때는 숫자형식의 movie_id값이 필요하기 때문에 위와 같이 변경해 주었다, 

In [410]:
csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape=(num_user, num_movie))
csr_data

<6040x3883 sparse matrix of type '<class 'numpy.longlong'>'
	with 834213 stored elements in Compressed Sparse Row format>

## 5. AlternatingLeastSquares 모델 훈련

In [411]:
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [412]:
csr_data_transpose = csr_data.T
csr_data_transpose

<3883x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 834213 stored elements in Compressed Sparse Column format>

In [413]:
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 6. 훈련된 모델이 예측한 나의 선호도 파악
-----
 * 내가 선호하는 5가지 영화와 그 외의 영화 하나를 골라 수행하기

### 내가 선호하는 영화 5개의 선호도

In [441]:
for movie in favorites5:
    my, favorite = user_to_idx[6041], movie_to_idx[movie['title']]
    my_vector, favorite_vector = als_model.user_factors[my], als_model.item_factors[favorite]
    print(movie['title'], np.dot(my_vector, favorite_vector))

Godfather, The 0.6422356
Matrix, The 0.55299544
Toy Story 0.4811591
Terminator 2: Judgment Day 0.5855664
Gladiator 0.5992232


### 그 외의 영화의 선호도

In [445]:
slaughterhouse = movie_to_idx['Slaughterhouse']
slaughterhouse_vector = als_model.item_factors[slaughterhouse]
np.dot(my_vector, slaughterhouse_vector)

0.005048998

## 7. 내가 좋아하는 영화와 비슷한 영화를 추천받기
-----

### 영화제목 입력시 인덱스 반환하는 딕셔너리 생성

In [436]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
print(idx_to_movie[67], movie_to_idx[idx_to_movie[67]])

Gladiator 67


### 좋아하는 영화와 비슷한 영화를 추천

In [456]:
favorite_movie = 'Gladiator'
movie_id = movie_to_idx[favorite_movie]
similar_movies = als_model.similar_items(movie_id, N=15)

for similar_movie in similar_movies:
    if similar_movie[1] >= 0.4811591 and movie_id != similar_movie[0]: # 자기자신 제거 및 가장 좋아하는 영화중 내적 값이 가장 낮은 것을 기준으로 선별  
        movie_name = idx_to_movie[similar_movie[0]]
        if movie_name[-5:] == ', The': # The가 붙는 영화이름 올바르게 정렬
            movie_name = movie_name[-3:] +' '+movie_name[:-5]
        print(movie_name)

The Patriot
Mission: Impossible 2
The Perfect Storm
U-571
X-Men
Frequency


> **좋아하는 영화를 선별할 때 내적 값이 0.4811591이상인 것을 선택하게 한 이유**   
   선호하는 영화 5개중 가장 내적값이 낮은 것이 0.4811591이었기에 이것을 **좋아할 만한 영화를 걸러내는 하한선**으로 삼는다면 보다 정확한 추천이 가능할 거라는 가설을 세우고 적용하였다. 

## 8. 내가 가장 좋아할 만한 영화들을 추천받기
-----

In [458]:
my_favorites = []

for favorite in favorites5:
    movie_id = movie_to_idx[favorite['title']]
    similar_movies = als_model.similar_items(movie_id, N=15)
    # 좋아하는 영화와 비슷한 영화 추천
    for similar_movie in similar_movies:
        if similar_movie[1] >= 0.4811591 and movie_id != similar_movie[0]:
            movie_name = idx_to_movie[similar_movie[0]]
            if movie_name[-5:] == ', The': # The가 붙는 영화이름 올바르게 정렬
                movie_name = movie_name[-3:] +' '+movie_name[:-5]
            my_favorites.append(movie_name)

my_favorites = list(set(my_favorites))
my_favorites

['Mission: Impossible 2',
 'Toy Story 2',
 'The Terminator',
 'Jurassic Park',
 'Total Recall',
 'The Patriot',
 'The Godfather: Part II',
 'Terminator 2: Judgment Day',
 'The Godfather: Part III',
 'The Matrix',
 'U-571',
 'The Fifth Element',
 'Face/Off',
 'Babe',
 'The Perfect Storm',
 'The Fugitive',
 'X-Men',
 'Groundhog Day',
 'Frequency',
 "Bug's Life, A",
 'Men in Black',
 'Aladdin']

## 9. 후기
-----

이번 노드를 진행하면서 가장 골치를 앓았던 부분은 csr matrix를 생성하는 부분이었다. 예제의 user_id나 artisit_id와는 달리 기본적으로 user_id와 movie_id가 정수형으로 되어 있어 별도의 변환이 불필요하다고 생각하고 그대로 진행했으나 계속 csr matrix 생성에 실패했다. csr matrix의 생성 원리를 이해하는 것부터 다시 진행하다보니 오후 시간이 전부 이 부분에 잡아먹히고 말았다. 결국 저녁에서야 예제처럼 인덱스를 부여하는 과정을 별도로 거치고 나니 이 부분은 해결되었다. 불필요하게 긴 시간동안 해맨 느낌이었지만 그외의 부분은 상대적으로 잘 진행되었기에 무사히 제출할 수 있었다. 

이번에 신경써서 했던 부분은 movie_id를 변경하는 부분과 좋아하는 영화를 추천하는 부분이었다. 특히 movie_id 변경은 앞서 언급한 src matrix 생성부분과 엮여있어 신경이 많이 쓰였다. 또한 영화를 추천하는 부분은 좋아하는 영화들의 내적값을 활용하여 추천의 정확성을 높이는 접근에 신경을 썼다. 

오늘도 노드를 하며 스트레스도 많이 받았지만 추천받은 결과를 보았을 때 유사한 장르의 영화들이 대부분이어서 보람이 있었고, 저번 해커톤때보다는 dataframe 사용에 더 익숙해진 것 같아 좋았다. 더 좋아지고 싶다.